- 本身browser是非同步的動作，先完成DOM後，才下載圖片，webdriver完全無法追蹤，所以必需要有等待的策略
- 必需等待網頁的DOM的完成，webdriver才可以取得元素，但有些情況是動態網頁(javascript所建立)，所以常會遇到沒有發現這個元素(**no such element**)

- 如以下範例(必需手動建立一個html網頁)

```
<!doctype html>
<meta charset=utf-8>
<title>Race Condition Example</title>

<script>
    var initialised = false;
    window.addEventListener("load", function () {
        var newElement = document.createElement("p");
        newElement.textContent = "Hello from JavaScript!";
        document.body.appendChild(newElement);
        initialised = true;
    });
</script>
```

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("file:///race_condition.html")
el = driver.find_element(By.TAG_NAME, "p")
assert el.text == "Hello from JavaScript!"

## 明確等待時間

In [ ]:
from selenium.webdriver.support.wait import WebDriverWait

def document_initialised(driver):
    return driver.execute_script("return initialised")

driver.navigate("file:///race_condition.html")
WebDriverWait(driver, timeout=10).until(document_initialised)
el = driver.find_element(By.TAG_NAME, "p")
assert el.text == "Hello from JavaScript!"

In [ ]:
from selenium.webdriver.support.wait import WebDriverWait

driver.navigate("file:///race_condition.html")
el = WebDriverWait(driver, timeout=3).until(lambda d: d.find_element(By.TAG_NAME,"p"))
assert el.text == "Hello from JavaScript!"

## 預設所有的等待時間

In [ ]:
driver.implicitly_wait(10)
driver.get("http://somedomain/url_that_delays_loading")
my_dynamic_element = driver.find_element(By.ID, "myDynamicElement")
